# Montage Management

In [ ]:
import re
# Import pyologger utilities
from pyologger.utils.folder_manager import *
from pyologger.plot_data.plotter import *
from pyologger.utils.montage_manager import MontageManager
from pyologger.utils.param_manager import ParamManager
from pyologger.load_data.datareader import DataReader
from pyologger.load_data.metadata import Metadata

# Load important file paths and configurations
config, data_dir, color_mapping_path, montage_path = load_configuration()

### Fetch metadata

Load in metadata stored in Notion databases. Alternatively, load in your own metadata in separate dataframes for deployments, loggers, recordings, animals, and datasets. See examples here in the `metadata_snapshot.pkl` file.

In [ ]:
from datetime import datetime, timedelta

overwrite = True
# Define the path to the metadata pickle file
metadata_pickle_path = os.path.join(data_dir, "00_Metadata/metadata_snapshot.pkl")

# Check if the metadata pickle file exists and is more than 5 days old
if overwrite or not os.path.exists(metadata_pickle_path) or (datetime.now() - datetime.fromtimestamp(os.path.getmtime(metadata_pickle_path))) > timedelta(days=5):
    
    metadata = Metadata()

    # Save database variables
    deployment_db = metadata.get_metadata("deployment_DB")
    logger_db = metadata.get_metadata("logger_DB")
    recording_db = metadata.get_metadata("recording_DB")
    animal_db = metadata.get_metadata("animal_DB")
    dataset_db = metadata.get_metadata("dataset_DB")
    procedure_db = metadata.get_metadata("procedure_DB")
    observation_db = metadata.get_metadata("observation_DB")
    collaborator_db = metadata.get_metadata("collaborator_DB")
    location_db = metadata.get_metadata("location_DB")
    montage_db = metadata.get_metadata("montage_DB")
    sensor_db = metadata.get_metadata("sensor_DB")
    attachment_db = metadata.get_metadata("attachment_DB")
    originalchannel_db = metadata.get_metadata("originalchannel_DB")
    standardizedchannel_db = metadata.get_metadata("standardizedchannel_DB")
    derivedsignal_db = metadata.get_metadata("derivedsignal_DB")
    derivedchannel_db = metadata.get_metadata("derivedchannel_DB")

    # Get the relations map
    relations_map = metadata.relations_map

    # Define the path to save the relations map
    relations_map_path = os.path.join(config['paths']['local_repo_path'], 'relations_map.json')

    # Save the relations map as a JSON file
    with open(relations_map_path, 'w') as file:
        json.dump(relations_map, file, indent=4)

    print(f"Relations map saved at: {relations_map_path}")

    ## OPTIONAL: Save metadata snapshot as a pickle file
    metadata.notion = None  # Temporarily remove the Notion client
    # Save metadata snapshot as a pickle file
    with open(metadata_pickle_path, "wb") as file:
        pickle.dump(metadata, file)

    print(f"Metadata snapshot saved at: {metadata_pickle_path}")
else:
    print(f"Recent metadata snapshot loaded, already present at: {metadata_pickle_path}")
    
    # Load the metadata snapshot from the pickle file
    with open(metadata_pickle_path, "rb") as file:
        metadata = pickle.load(file)
    
    # Save database variables
    deployment_db = metadata.get_metadata("deployment_DB")
    logger_db = metadata.get_metadata("logger_DB")
    recording_db = metadata.get_metadata("recording_DB")
    animal_db = metadata.get_metadata("animal_DB")
    dataset_db = metadata.get_metadata("dataset_DB")
    procedure_db = metadata.get_metadata("procedure_DB")
    observation_db = metadata.get_metadata("observation_DB")
    collaborator_db = metadata.get_metadata("collaborator_DB")
    location_db = metadata.get_metadata("location_DB")
    montage_db = metadata.get_metadata("montage_DB")
    sensor_db = metadata.get_metadata("sensor_DB")
    attachment_db = metadata.get_metadata("attachment_DB")
    originalchannel_db = metadata.get_metadata("originalchannel_DB")
    standardizedchannel_db = metadata.get_metadata("standardizedchannel_DB")
    derivedsignal_db = metadata.get_metadata("derivedsignal_DB")
    derivedchannel_db = metadata.get_metadata("derivedchannel_DB")


In [ ]:
# Select dataset folder
dataset_folder = select_folder(data_dir, "Select a dataset folder:")

In [ ]:
deployment_folder = select_folder(dataset_folder, "Select a deployment folder:")

In [ ]:
# Extract deployment_id and animal_id from the folder name
match = re.match(r"(\d{4}-\d{2}-\d{2}_[a-z]{4}-\d{3})", os.path.basename(deployment_folder), re.IGNORECASE)
if match:
    deployment_id = match.group(1)  # Extract YYYY-MM-DD_animalID
    animal_id = deployment_id.split("_")[1]  # Extract animal ID
    print(f"✅ Extracted deployment ID: {deployment_id}, Animal ID: {animal_id}")
else:
    raise ValueError(f"❌ Unable to extract deployment ID from folder: {deployment_folder}")

## Read Files in Deployment Folder

Using datareader to load in files


In [ ]:
# Print extracted values for debugging
print(f"🐳 Deployment ID: {deployment_id}, Animal ID: {animal_id}")

deployment_info, loggers_used = metadata.extract_essential_metadata(deployment_id)

In [ ]:
# Step 4: Initialize DataReader with dataset folder, deployment ID, and optional data subfolder
data_pkl = DataReader(dataset_folder=dataset_folder, deployment_id=deployment_id, data_subfolder="01_raw-data", montage_path=montage_path)
# Step 5: Initialize config manager
param_manager = ParamManager(deployment_folder=deployment_folder, deployment_id=deployment_id)

## Preview montages

In [ ]:
original_channels = originalchannel_db[originalchannel_db['Montages'].str.contains('hr-montage_V2', na=False)]
original_channels

In [ ]:
use_csv = False # 🔁 Set this to True if you want to load montage from CSVs
csv_folder = "/path/to/csvs"  # Folder where your CSVs are stored

montage_inputs = {}

standardizedchannel_db = standardizedchannel_db.copy()
standardizedchannel_db.loc[:, "Color"] = standardizedchannel_db["Color preview"].str.extract(
    r'\\color\s*\{\s*#?(\w+)\s*\}')

for logger in loggers_used:
    logger_id = logger["Logger ID"]
    montage_id = logger["Montage ID"]

    if use_csv:
        csv_path = os.path.join(csv_folder, f"{montage_id}.csv")
        if os.path.exists(csv_path):
            montage_df = pd.read_csv(csv_path)
            montage_inputs[logger_id] = montage_df
            print(f"📄 Loaded montage CSV for Logger: {logger_id} (Montage ID: {montage_id})")
        else:
            print(f"❌ CSV not found for montage ID: {montage_id} (Logger: {logger_id})")
        continue

    # Otherwise, build from original + standardized DBs
    original_channels = originalchannel_db[originalchannel_db['Montages'].str.contains(montage_id, na=False)]

    if original_channels.empty:
        print(f"⚠️ No original channels found for montage ID: {montage_id} (Logger: {logger_id})")
        continue

    montage_df = pd.merge(
        standardizedchannel_db,
        original_channels,
        on='Standardized Channel ID',
        suffixes=('', '_original')
    )

    montage_df.columns = montage_df.columns.str.lower().str.replace(' ', '_')
    montage_df = montage_df[[
        'original_channel_id', 'original_unit', 'manufacturer_sensor_name',
        'standardized_channel_id', 'standardized_unit', 'standardized_sensor_type'
    ]]

    montage_inputs[logger_id] = montage_df
    print(f"✅ Created montage_df for Logger: {logger_id} (Montage ID: {montage_id})")


In [ ]:
montage_manager = MontageManager(montage_folder=os.path.dirname(montage_path))

montages_metadata = montage_manager.add_missing_montages_per_logger(
	loggers_used=loggers_used,
	montage_inputs=montage_inputs
)